In [ ]:
pip install langchain langchain-community ##python-dotenv


In [1]:
from langchain_community.chat_models import ChatOllama
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
#from dotenv import load_dotenv
import json


In [2]:
llm = ChatOllama(model="llama3:latest")

C:\Users\gsaba\AppData\Local\Temp\ipykernel_12368\3566182012.py:1: LangChainDeprecationWarning: The class `ChatOllama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import ChatOllama``.
  llm = ChatOllama(model="llama3:latest ")


In [3]:


# 1. Tender Summary Agent
def summarize_tender(file_path):
    with open(file_path, 'r') as file:
        tender_data = file.read()

    prompt = PromptTemplate.from_template(
        "You are a procurement expert. Summarize the tender below in bullet points:\n\n{tender_text}"
    )
    chain = LLMChain(llm=llm, prompt=prompt)
    return chain.run({"tender_text": tender_data})


# 2. Product Brief Agent
def summarize_product(file_path):
    with open(file_path, 'r') as file:
        product_data = file.read()

    prompt = PromptTemplate.from_template(
        "You are a technical analyst. Summarize the product brief below for a buyer:\n\n{product_text}"
    )
    chain = LLMChain(llm=llm, prompt=prompt)
    return chain.run({"product_text": product_data})


# 3. Negotiation Strategy Agent
# def recommend_strategy(tender_path, perf_path):
#     with open(tender_path, 'r') as f:
#         tender = f.read()
#     with open(perf_path, 'r') as f:
#         perf = json.load(f)

#     perf_text = "\n".join(
#         [f"{k}: delivery={v['delivery_rating']}, quality={v['quality_rating']}, price={v['pricing_avg']}" 
#          for k, v in perf.items()]
#     )

#     prompt = PromptTemplate.from_template(
#         "You are a negotiation strategist. Given the tender:\n\n{tender}\n\n"
#         "And supplier performance data:\n\n{performance}\n\n"
#         "Suggest negotiation strategy in bullet points."
#     )
#     chain = LLMChain(llm=llm, prompt=prompt)
#     return chain.run({"tender": tender, "performance": perf_text})

def recommend_strategy_multi(tender_path, perf_path, bids_path):
    with open(tender_path, 'r') as f:
        tender = f.read()
    with open(perf_path, 'r') as f:
        perf = json.load(f)
    with open(bids_path, 'r') as f:
        bids = json.load(f)

    perf_text = "\n".join(
        [f"{k}: delivery={v['delivery_rating']}, quality={v['quality_rating']}, price={v['pricing_avg']}" 
         for k, v in perf.items()]
    )
    
    bids_text = "\n".join(
        [f"{k}: price={v['price_per_unit']}, quantity={v['quantity_offered']}, lead_time={v['lead_time_days']} days"
         for k, v in bids.items()]
    )

    prompt = PromptTemplate.from_template(
        "You're a negotiation advisor. A buyer received the tender:\n\n{tender}\n\n"
        "Supplier performance:\n{performance}\n\n"
        "Bids received:\n{bids}\n\n"
        "Evaluate and suggest negotiation recommendations."
    )
    
    chain = LLMChain(llm=llm, prompt=prompt)
    return chain.run({"tender": tender, "performance": perf_text, "bids": bids_text})



# 4. Draft Email Agent
def draft_negotiation_email(tender_path, strategy):
    with open(tender_path, 'r') as f:
        tender = f.read()

    prompt = PromptTemplate.from_template(
        "You are a professional buyer. Based on the tender:\n\n{tender}\n\n"
        "And negotiation strategy:\n\n{strategy}\n\n"
        "Draft a negotiation email to the supplier."
    )
    chain = LLMChain(llm=llm, prompt=prompt)
    return chain.run({"tender": tender, "strategy": strategy})



In [5]:


# ✅ Run the Flow
if __name__ == "__main__":
    tender_summary = summarize_tender("tender_001.txt")
    product_summary = summarize_product("product_brief_001.txt")
    strategy = recommend_strategy_multi("tender_001.txt", "supplier_perf.json","bids.json")
    draft_email = draft_negotiation_email("tender_001.txt", strategy)

    print("📄 Tender Summary:\n", tender_summary)
    print("\n📦 Product Brief:\n", product_summary)
    print("\n🧠 Negotiation Strategy:\n", strategy)
    print("\n✉️ Draft Email:\n", draft_email)

📄 Tender Summary:
 Here is a summary of the tender in bullet points:

• Tender ID: RFP2025-012
• Product: Industrial Refrigeration Units
• Quantity: 250 units
• Required Delivery: Within 6 weeks
• Payment Terms: Net 60 (payment expected within 60 days)
• Submission Deadline: April 30, 2025
• Preferred Vendors: ABC Cooling Inc and ThermoTech LLC

📦 Product Brief:
 Here's a summary of the product brief for a buyer:

**Product:** Industrial Refrigeration Unit

**Key Specifications:**

* Power consumption: 10kW
* Temperature range: -10Â°C to +5Â°C (suitable for various industrial applications)

**Compliance and Certifications:**

* ISO 9001 quality management standard certified
* CE marked, ensuring compliance with European health, safety, and environmental protection directives

**Warranty:** 3-year warranty for peace of mind and reliable operation

This summary provides a concise overview of the product's key features, certifications, and warranty, making it easy for buyers to quickly as